In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [8]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
#indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

# pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
# pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

pairs_train = [x for x in pairs if int(x[0].split('_')[-1]) <= 9]
pairs_test = [x for x in pairs if int(x[0].split('_')[-1]) >= 10]

In [9]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [11]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
encoder_output = word_embeddings + sinusoidal_embeddings

for i in range(1):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 1s - loss: 3.4102 - val_loss: 3.3886 - 1s/epoch - 1s/step
Epoch 2/2000
1/1 - 0s - loss: 3.3858 - val_loss: 3.3677 - 66ms/epoch - 66ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.3657 - val_loss: 3.3479 - 74ms/epoch - 74ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.3465 - val_loss: 3.3291 - 69ms/epoch - 69ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.3280 - val_loss: 3.3110 - 78ms/epoch - 78ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.3100 - val_loss: 3.2939 - 73ms/epoch - 73ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.2928 - val_loss: 3.2775 - 67ms/epoch - 67ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.2764 - val_loss: 3.2618 - 69ms/epoch - 69ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.2606 - val_loss: 3.2467 - 77ms/epoch - 77ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.2453 - val_loss: 3.2323 - 75ms/epoch - 75ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.2308 - val_loss: 3.2187 - 80ms/epoch - 80ms/step
Epoch 12/2000
1/1 - 0s - loss: 3.2169 - val_loss: 3.2058 - 77ms/epoch - 77ms/step
Epoch 13/2000
1/1 - 0s - loss

1/1 - 0s - loss: 2.9154 - val_loss: 2.9317 - 69ms/epoch - 69ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.9150 - val_loss: 2.9314 - 68ms/epoch - 68ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.9147 - val_loss: 2.9312 - 55ms/epoch - 55ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.9143 - val_loss: 2.9309 - 74ms/epoch - 74ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.9140 - val_loss: 2.9306 - 64ms/epoch - 64ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.9136 - val_loss: 2.9304 - 73ms/epoch - 73ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.9132 - val_loss: 2.9301 - 68ms/epoch - 68ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.9129 - val_loss: 2.9299 - 57ms/epoch - 57ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.9126 - val_loss: 2.9296 - 73ms/epoch - 73ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.9122 - val_loss: 2.9294 - 59ms/epoch - 59ms/step
Epoch 111/2000
1/1 - 0s - loss: 2.9119 - val_loss: 2.9291 - 69ms/epoch - 69ms/step
Epoch 112/2000
1/1 - 0s - loss: 2.9115 - val_loss: 2.9289 - 68ms/epoch - 68ms/step
Epoch 113/2000
1/1 

Epoch 200/2000
1/1 - 0s - loss: 2.7533 - val_loss: 2.7865 - 65ms/epoch - 65ms/step
Epoch 201/2000
1/1 - 0s - loss: 2.7456 - val_loss: 2.7822 - 58ms/epoch - 58ms/step
Epoch 202/2000
1/1 - 0s - loss: 2.7410 - val_loss: 2.7761 - 56ms/epoch - 56ms/step
Epoch 203/2000
1/1 - 0s - loss: 2.7348 - val_loss: 2.7713 - 75ms/epoch - 75ms/step
Epoch 204/2000
1/1 - 0s - loss: 2.7302 - val_loss: 2.7666 - 59ms/epoch - 59ms/step
Epoch 205/2000
1/1 - 0s - loss: 2.7255 - val_loss: 2.7601 - 83ms/epoch - 83ms/step
Epoch 206/2000
1/1 - 0s - loss: 2.7190 - val_loss: 2.7541 - 61ms/epoch - 61ms/step
Epoch 207/2000
1/1 - 0s - loss: 2.7131 - val_loss: 2.7491 - 60ms/epoch - 60ms/step
Epoch 208/2000
1/1 - 0s - loss: 2.7083 - val_loss: 2.7433 - 61ms/epoch - 61ms/step
Epoch 209/2000
1/1 - 0s - loss: 2.7023 - val_loss: 2.7373 - 61ms/epoch - 61ms/step
Epoch 210/2000
1/1 - 0s - loss: 2.6958 - val_loss: 2.7329 - 60ms/epoch - 60ms/step
Epoch 211/2000
1/1 - 0s - loss: 2.6908 - val_loss: 2.7283 - 59ms/epoch - 59ms/step
Epoc

Epoch 299/2000
1/1 - 0s - loss: 2.3080 - val_loss: 2.3234 - 76ms/epoch - 76ms/step
Epoch 300/2000
1/1 - 0s - loss: 2.3043 - val_loss: 2.3203 - 74ms/epoch - 74ms/step
Epoch 301/2000
1/1 - 0s - loss: 2.3013 - val_loss: 2.3166 - 76ms/epoch - 76ms/step
Epoch 302/2000
1/1 - 0s - loss: 2.2981 - val_loss: 2.3130 - 82ms/epoch - 82ms/step
Epoch 303/2000
1/1 - 0s - loss: 2.2943 - val_loss: 2.3084 - 85ms/epoch - 85ms/step
Epoch 304/2000
1/1 - 0s - loss: 2.2900 - val_loss: 2.3048 - 81ms/epoch - 81ms/step
Epoch 305/2000
1/1 - 0s - loss: 2.2864 - val_loss: 2.3027 - 78ms/epoch - 78ms/step
Epoch 306/2000
1/1 - 0s - loss: 2.2846 - val_loss: 2.3037 - 79ms/epoch - 79ms/step
Epoch 307/2000
1/1 - 0s - loss: 2.2856 - val_loss: 2.3064 - 104ms/epoch - 104ms/step
Epoch 308/2000
1/1 - 0s - loss: 2.2894 - val_loss: 2.2949 - 69ms/epoch - 69ms/step
Epoch 309/2000
1/1 - 0s - loss: 2.2767 - val_loss: 2.2899 - 74ms/epoch - 74ms/step
Epoch 310/2000
1/1 - 0s - loss: 2.2726 - val_loss: 2.2908 - 68ms/epoch - 68ms/step
Ep

In [12]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [13]:
(np.mean(acc), np.mean(prob))

(0.0, 0.0027104116)

In [14]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
sinusoidal_embeddings = get_sinusoidal_embeddings(len(x_masked_train[0]), embed_dim)
encoder_output = word_embeddings + sinusoidal_embeddings

for i in range(1):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 1s - loss: 3.7518 - val_loss: 3.3174 - 1s/epoch - 1s/step
Epoch 2/2000
1/1 - 0s - loss: 3.3064 - val_loss: 3.1280 - 181ms/epoch - 181ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.1152 - val_loss: 3.0649 - 187ms/epoch - 187ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.0495 - val_loss: 3.0413 - 172ms/epoch - 172ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.0222 - val_loss: 3.0265 - 175ms/epoch - 175ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.0036 - val_loss: 3.0128 - 196ms/epoch - 196ms/step
Epoch 7/2000
1/1 - 0s - loss: 2.9867 - val_loss: 2.9943 - 206ms/epoch - 206ms/step
Epoch 8/2000
1/1 - 0s - loss: 2.9660 - val_loss: 2.9721 - 190ms/epoch - 190ms/step
Epoch 9/2000
1/1 - 0s - loss: 2.9425 - val_loss: 2.9519 - 161ms/epoch - 161ms/step
Epoch 10/2000
1/1 - 0s - loss: 2.9212 - val_loss: 2.9382 - 123ms/epoch - 123ms/step
Epoch 11/2000
1/1 - 0s - loss: 2.9065 - val_loss: 2.9314 - 137ms/epoch - 137ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.8984 - val_loss: 2.9286 - 136ms/epoch - 136ms/step
Epoch 1

Epoch 99/2000
1/1 - 0s - loss: 1.9941 - val_loss: 2.0517 - 176ms/epoch - 176ms/step
Epoch 100/2000
1/1 - 0s - loss: 1.9916 - val_loss: 2.0493 - 180ms/epoch - 180ms/step
Epoch 101/2000
1/1 - 0s - loss: 1.9896 - val_loss: 2.0473 - 130ms/epoch - 130ms/step
Epoch 102/2000
1/1 - 0s - loss: 1.9877 - val_loss: 2.0452 - 143ms/epoch - 143ms/step
Epoch 103/2000
1/1 - 0s - loss: 1.9861 - val_loss: 2.0430 - 140ms/epoch - 140ms/step
Epoch 104/2000
1/1 - 0s - loss: 1.9846 - val_loss: 2.0411 - 129ms/epoch - 129ms/step
Epoch 105/2000
1/1 - 0s - loss: 1.9833 - val_loss: 2.0397 - 138ms/epoch - 138ms/step
Epoch 106/2000
1/1 - 0s - loss: 1.9821 - val_loss: 2.0383 - 150ms/epoch - 150ms/step
Epoch 107/2000
1/1 - 0s - loss: 1.9810 - val_loss: 2.0372 - 159ms/epoch - 159ms/step
Epoch 108/2000
1/1 - 0s - loss: 1.9801 - val_loss: 2.0364 - 128ms/epoch - 128ms/step
Epoch 109/2000
1/1 - 0s - loss: 1.9792 - val_loss: 2.0358 - 131ms/epoch - 131ms/step
Epoch 110/2000
1/1 - 0s - loss: 1.9784 - val_loss: 2.0350 - 130ms/

In [15]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [16]:
(np.mean(acc), np.mean(prob))

(0.0, 0.003079601)

In [17]:
x_masked_test

array([[10,  0,  1, 10],
       [10,  0,  2, 10],
       [10,  0,  3, 10],
       ...,
       [19, 18, 15, 19],
       [19, 18, 16, 19],
       [19, 18, 17, 19]])